In [21]:
import autograd.numpy as anp
from numpy import random as rnd
from numpy import linalg as la 

import pymanopt
from pymanopt.manifolds import Grassmann
from pymanopt.solvers import TrustRegions

This example generates a random 128 x 128 symmetric matrix, and finds the dominant invariant 3-dimensional subspace for this matrix.
That is, it finds the subspace spanned by the three eigenvectors with the largest eigenvalues

In [17]:
def create_cost_egrad_ehess(manifold, matrix):
	egrad = ehess = None
	@pymanopt.function.Autograd
	def cost(X):
		return -anp.trace(X.T @ matrix @ X)
	return cost, egrad, ehess

In [18]:
num_rows = 128
subspace_dimension = 3
matrix = rnd.randn(num_rows, num_rows)
matrix = 0.5* (matrix + matrix.T)

In [19]:
manifold = Grassmann(num_rows, subspace_dimension)
cost, egrad, ehess = create_cost_egrad_ehess(manifold, matrix)
problem = pymanopt.Problem(manifold, cost=cost, egrad=egrad, ehess=ehess)

In [20]:
solver = TrustRegions()
estimated_spanning_set = solver.solve(problem, Delta_bar=8*anp.sqrt(subspace_dimension))

Optimizing...
                                            f: +3.669090e+00   |grad|: 2.698056e+01
acc       k:     1     num_inner:     0     f: -2.224805e+01   |grad|: 2.008826e+01   negative curvature
acc       k:     2     num_inner:     1     f: -3.184592e+01   |grad|: 2.040852e+01   negative curvature
acc       k:     3     num_inner:     1     f: -3.756073e+01   |grad|: 1.578689e+01   exceeded trust region
acc       k:     4     num_inner:     2     f: -4.194086e+01   |grad|: 9.289922e+00   exceeded trust region
acc       k:     5     num_inner:     4     f: -4.403647e+01   |grad|: 5.319894e+00   exceeded trust region
acc       k:     6     num_inner:     4     f: -4.513197e+01   |grad|: 5.013814e+00   negative curvature
acc       k:     7     num_inner:     8     f: -4.604457e+01   |grad|: 7.240103e-01   reached target residual-kappa (linear)
acc       k:     8     num_inner:     6     f: -4.609028e+01   |grad|: 6.981344e-02   reached target residual-kappa (linear)
acc       k: 

In [23]:
eigenvalues, eigenvectors = la.eig(matrix)
column_indices = anp.argsort(eigenvalues)[-subspace_dimension:]
spanning_set = eigenvectors[:, column_indices]
print(
	"Geodesic distance between true and estimated dominant subspace:",
	manifold.dist(spanning_set, estimated_spanning_set),
)


Geodesic distance between true and estimated dominant subspace: 4.2146848510894035e-08


In [27]:
spanning_set.shape

(128, 3)